In [85]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import os
import urllib
from scipy.stats import ttest_ind
url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/live/us-counties.csv"
file_path = os.path.join("data", "covidCounty")
os.makedirs(file_path, exist_ok=True)
csv_path = os.path.join(file_path, "Corona-counties.csv")
urllib.request.urlretrieve(url, csv_path)
url2 = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv"
file_path2 = os.path.join("data", "popCounty")
os.makedirs(file_path2, exist_ok=True)
csv_path2 = os.path.join(file_path2, "Corona-counties.csv")
urllib.request.urlretrieve(url2, csv_path2)

('data/popCounty/Corona-counties.csv',
 <http.client.HTTPMessage at 0x1224a6250>)

In [148]:
df = pd.read_csv(csv_path)
df2 = pd.read_csv("data/2020-q3-county-median-home-prices-by-price-01-06-2020.csv", skiprows=[1,2,3,4,5,3125,3126,3127,3128])
df3 = pd.read_csv(csv_path2)
df2.rename(columns={"Unnamed: 0":"County","Unnamed: 1":"State", "Unnamed: 3": "Price","Unnamed: 5":"Monthly(2.76)","Unnamed: 7":"Monthly(3.73)"},inplace=True)
df2.drop(columns=["Unnamed: 2","Monthly","Monthly.1"],inplace=True)
df2["County"] = df2.apply(lambda x:re.findall("(.+?)(?=( County,$| city,$| Parish,$| Borough,$|,$))",x["County"])[0][0], axis=1)
df["Name"] = df.apply(lambda x: x["county"]+", "+x["state"], axis=1)
df2["Name"] = df2.apply(lambda x: x["County"]+", "+x["State"], axis=1)
df.set_index("Name",inplace=True)
df2.set_index("Name",inplace=True)
mergedH = pd.merge(df2,df,how="inner",left_index=True,right_index=True)
df3.drop(df3.columns.difference(['POPESTIMATE2019','CTYNAME',"STNAME"]), 1, inplace=True)
df3["CTYNAME"] = df3.apply(lambda x:re.findall("(.+?)(?=( County$| city$| Parish$| Borough$|$))",x["CTYNAME"])[0][0], axis=1)
df3["Name"] = df3.apply(lambda x: x["CTYNAME"]+", "+x["STNAME"], axis=1)
df3.set_index("Name", inplace=True)
merged=pd.merge(mergedH,df3,how="inner",left_index=True,right_index=True)
merged.drop(merged.columns.difference(["Price",'Monthly(2.76)',"Monthly(3.73)","cases","deaths"]), 1, inplace=True)
merged["rate"] = merged.apply(lambda x: (x["deaths"]/x["cases"])*100, axis=1)
#pd.to_numeric(merged["Price"])
merged["Price"] = merged.apply(lambda x: int(x["Price"].replace(',', '')), axis=1)
merged = merged[merged.cases>2000]
print(ttest_ind(merged["Price"], merged["rate"]))
sortedF = merged.sort_values(by=['rate'])


Ttest_indResult(statistic=67.29073024606738, pvalue=0.0)


In [120]:
print(df2.shape)
print(df.shape)
print(df3.shape)

(3119, 5)
(3246, 10)
(3193, 3)


In [149]:
sortedF

,Price,Monthly(2.76),Monthly(3.73),cases,deaths,rate
Name,,,,,,
"Summit, Utah",677700,"2,460","2,670",4899,8.0,0.163299
"Ford, Kansas",115300,420,470,5543,10.0,0.180408
"Montgomery, Kansas",78100,280,320,3218,6.0,0.186451
"Latah, Idaho",244000,890,950,2643,6.0,0.227015
"Cache, Utah",294200,"1,070","1,160",14648,34.0,0.232114
...,...,...,...,...,...,...
"Tuscola, Michigan",107900,390,430,3133,134.0,4.277051
"Hockley, Texas",97000,350,390,2290,98.0,4.279476
"Neshoba, Mississippi",84700,310,350,3734,165.0,4.418854
